# Vigenér cipher decryption
---

In [1]:
import os
import sys

__module_path = os.path.abspath(os.path.join('..'))
if __module_path not in sys.path:
    sys.path.append(__module_path)

import cryptoanalysis

import numpy as np

from IPython.display import Javascript, HTML

from plotly import graph_objs, __version__
import plotly.offline as plotly

from ipywidgets import Layout, widgets, Box, HBox, VBox

In [2]:
%matplotlib inline

In [3]:
plotly_version = __version__

# For online connection:
# plotly.tools.set_credentials_file(username='CermakM', api_key='q2yoGYw052dyHMd8ztqx')
# plotly.tools.set_config_file(world_readable=False, sharing='secret')

plotly.init_notebook_mode(connected=True)

In [4]:
_submitted = False
_reset = True

cipher = ''
decrypted_text = ''

analyser = cryptoanalysis.decryption.Analyser(lang='en')

### Encrypted text

In [5]:
def _create_text_box(*args, **kwargs):
    for widget in args:
        widget.layout=Layout(width='100%', height='200px')
    box = Box(args, **kwargs)
    box.layout.display = 'flex'
    box.layout.align_items = 'stretch'
    box.layout.min_height = '230px'
    box.layout.flex_direction = 'row'
    return box

In [6]:
input_area = widgets.Textarea(
    placeholder='Encrypted text',
)  

output_area = widgets.Textarea(
    placeholder='Decrypted text',
    disabled=True
)

input_box = _create_text_box(input_area)
output_box = _create_text_box(output_area)

In [7]:
submit_input_button = widgets.Button(
    description='SUBMIT',
    disabled=False,
    button_style='success',
    icon='check'
)

reset_input_button = widgets.Button(
    description='RESET',
    disabled=False,
    button_style='danger'
)

In [8]:
def _on_submit(sender):
    global analyser, cipher, _submitted, _reset
            
    cipher = input_area.value
    
    if not cipher:
        return
    
    _reset = False

    analyser = cryptoanalysis.decryption.Analyser(cipher=cipher, lang='en')
    
    display(Javascript('IPython.notebook.execute_cells_below()'))
    
    if not _submitted:
#         Javascript('$.notify("Cipher submitted");')
        
        _submitted = True
    
def _on_reset(sender):
    global analyser, cipher, _submitted
    
    _submitted = False
    
    cipher = ''
    analyser = cryptoanalysis.decryption.Analyser(lang='en')
    input_area.value = ''
    
    display(Javascript('IPython.notebook.execute_cells_below()'))
    
    if not _reset:
#         Javascript('$.notify("Analyser has been reset")')
        
        _reset = True

In [9]:
submit_input_button.on_click(_on_submit)
reset_input_button.on_click(_on_reset)

In [10]:
submit_input_button.layout.margin = '2px 2px 2px auto'
reset_input_button.layout.margin = '2px 2px 2px 5px'

submit_box = HBox([submit_input_button, reset_input_button])

submit_box.layout.display = 'flex'
submit_box.layout.flex_flow = 'row'
submit_box.layout.align_items = 'flex-end'

In [11]:
button_samples = widgets.ToggleButtons(
    options=['Sample 1', 'Sample 2', 'Sample 3'],
    description='Load sample:',
    disabled=False,
    button_style='',
    tooltips=['Caesar cipher', 'Vigener cipher - short', 'Vigener cipher - long']
)

button_load = widgets.Button(
    description='LOAD',
    disabled=False,
    button_style='success',
    icon='upload'
)

samples = ['samples/TIKcipher1.txt', 'samples/TIKcipher2a.txt', 'samples/TIKcipher2b.txt']

def _on_sample_load(sender):
    sample_to_read = samples[button_samples.index]
    with open(os.path.join(__module_path, sample_to_read)) as s:
        input_area.value = s.read()
        
button_load.on_click(_on_sample_load)

In [12]:
def _on_input(sender):
    pass
    
input_area.observe(_on_input, names='value')

In [13]:
display(HBox([button_samples, button_load]))

A Jupyter Widget

In [27]:
display(VBox([input_box, submit_box]))

<IPython.core.display.Javascript object>

A Jupyter Widget

---
### Frequency Analysis

In [28]:
default_lang_trace = graph_objs.Bar(
    x=analyser.alphabet,
    y=analyser.letter_frequency,
    name='Language'
)

if cipher:
    cipher_dict = analyser.get_char_frequency()
    cipher_trace = graph_objs.Bar(
        x=list(cipher_dict.keys()),
        y=list(cipher_dict.values()),
        name='Cipher'
    )
    data = [default_lang_trace, cipher_trace]

else:
    data = [default_lang_trace]
    
layout = graph_objs.Layout(
    showlegend=True,
    title='Frequency analysis',
    xaxis=dict(tickangle=-45),
    yaxis=dict(
        title='Letter frequency',
        tickformat=' %'),
    barmode='group'
)


In [29]:
fig = graph_objs.Figure(data=data, layout=layout)

plotly.iplot(fig)

---
### Key decryption

In [30]:
options = [] if not cipher else analyser.get_key_len_list()

widget_key_len = widgets.ToggleButtons(
    options=['-'] if not options else options,
    disabled=not options,
    button_style=''
)

key = ''

def _on_len_change(sender):
    global key
    if _submitted:
        key, _ = analyser.get_keys(key_id=widget_key_len.index)
        output_area.value = analyser.decipher(custom_key=key, rot=widget_rotation.value)        
    else:
        key = ''

widget_key_len.observe(_on_len_change, 'value')

_on_len_change(None)  # Run for initialization

In [31]:
display(widgets.Label(value="Suggested key length:"))
display(widget_key_len)

A Jupyter Widget

A Jupyter Widget

#### Decrypted key

In [32]:
pseudobutton_key = widgets.ToggleButtons(
    options=[key or '-'],
    button_style='',
    disabled=True
)

In [33]:
display(widgets.Label(value="Decrypted key:"))
display(pseudobutton_key)

A Jupyter Widget

A Jupyter Widget

---
### Text decryption

In [34]:
widget_rotation = widgets.ToggleButtons(
    options=[1, 0],
    disabled=False,
    tooltips=['Rotation 1: `a` -> `b`', 'Rotation 0: `a` -> `a`']
)


In [35]:
def _on_rot_change(sender):

    output_area.value = analyser.decipher(custom_key=key, rot=widget_rotation.value)

In [36]:
widget_rotation.observe(_on_rot_change, 'value')

---

In [37]:
display(widgets.Label(value="Select rotation:"))
display(widget_rotation)

A Jupyter Widget

A Jupyter Widget

In [38]:
result = ''
if _submitted:
    result = analyser.decipher(custom_key=key, rot=widget_rotation.value)
    
output_area.value = result

In [39]:
display(widgets.Label(value="Decrypted text:"))
display(_create_text_box(output_area))

A Jupyter Widget

A Jupyter Widget